In [3]:
import numpy as np

#759x9
loaded_data = np.loadtxt('/content/drive/MyDrive/Colab Notebooks/diabetes.csv',delimiter=',')

x_train_np = loaded_data[:,0:-1]
y_train_np = loaded_data[:,[-1]]

print(loaded_data.shape)
print(x_train_np.shape)
print(y_train_np.shape)




(759, 9)
(759, 8)
(759, 1)


In [4]:
import torch
from torch import nn

#numpy array를 pytorch의 tensor
x_train = torch.Tensor(x_train_np)
y_train = torch.Tensor(y_train_np)



In [27]:
#nn.Module을 상속해야함
class MyLogisticRegressionModel(nn.Module):
  def __init__(self):
    super().__init__()

    #Layer stack (=Model) 구성
    self.logistic_stack = nn.Sequential(
        #(입력노드,출력노드)
        nn.Linear(8,1),
        nn.Sigmoid()
    )

  def forward(self,data):
    prediction = self.logistic_stack(data)

    return prediction

In [28]:
model = MyLogisticRegressionModel()

for param in model.parameters():
  #calculated w and b
  print(param)

Parameter containing:
tensor([[ 0.2693,  0.2735,  0.1776, -0.0496,  0.1604, -0.3194, -0.2606,  0.2719]],
       requires_grad=True)
Parameter containing:
tensor([-0.0324], requires_grad=True)


In [29]:
loss_function = nn.BCELoss()#데이터에 맞는 손실함수 선택
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1) #데이터에 맞는 optimizer ex) Gradient Decent, Stochastic Gradient Descent

train_loss_list =[]
train_accuracy_list=[]

nums_epoch = 5000

for epoch in range(nums_epoch+1):
  outputs = model(x_train) # 전체결과 값 추출
  loss = loss_function(outputs,y_train) #전체결과값,정답데이터들 엮어서 로스계산
  train_loss_list.append(loss.item()) #loss가 아니고 loss.item()

  predictions = outputs>0.5 #텐서는 이런연산이 가능한가 보네?

  correct = (predictions.float() == y_train)
  accuracy = correct.sum().item()/len(correct)

  train_accuracy_list.append(accuracy)

  optimizer.zero_grad() #grad초기화를 꼭 해야하나?
  loss.backward() #loss_function이랑 My~~Model이랑 마운트한적이 없는데 어떻게 back-propaganation이 되는거임?
  optimizer.step()#얘는 model.parameters()를 넣어줬으니까 그렇다 쳐도?

  if epoch%100 == 0:
    print( int((epoch/nums_epoch)*100),'%', 'current loss :',loss.item(),'accuracy = ',accuracy)

0 % current loss : 0.7347491979598999 accuracy =  0.37549407114624506
2 % current loss : 0.5831705927848816 accuracy =  0.6600790513833992
4 % current loss : 0.5432320833206177 accuracy =  0.7193675889328063
6 % current loss : 0.5208613276481628 accuracy =  0.7536231884057971
8 % current loss : 0.5071917176246643 accuracy =  0.7628458498023716
10 % current loss : 0.4982457756996155 accuracy =  0.766798418972332
12 % current loss : 0.49208569526672363 accuracy =  0.7720685111989459
14 % current loss : 0.4876808822154999 accuracy =  0.769433465085639
16 % current loss : 0.4844399690628052 accuracy =  0.7681159420289855
18 % current loss : 0.48200178146362305 accuracy =  0.7654808959156785
20 % current loss : 0.48013409972190857 accuracy =  0.766798418972332
22 % current loss : 0.4786817133426666 accuracy =  0.7707509881422925
24 % current loss : 0.47753748297691345 accuracy =  0.769433465085639
26 % current loss : 0.47662562131881714 accuracy =  0.769433465085639
28 % current loss : 0.47

**Validation Data**

train data를 통해 학습을 완료한 후 Overfitting을 체크할 수 있는 data
**Test Data**

Traindata가 아닌 모델 성능을 검증할 수 있는 test data